# Mobile Carrier Customer Service Agent w/Account Lookup


In [1]:
from typing import Annotated, Dict
from pydantic import BaseModel
from semantic_kernel import Kernel
from semantic_kernel.filters import FunctionInvocationContext
from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, OpenAIChatPromptExecutionSettings
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.functions import kernel_function, KernelArguments
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.contents import ChatMessageContent
from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.functions import kernel_function
from pydantic import BaseModel
from datetime import datetime
import uuid

In [2]:
# Define the auto function invocation filter that will be used by the kernel
async def function_invocation_filter(context: FunctionInvocationContext, next):
    """A filter that will be called for each function call in the response."""
    if "messages" not in context.arguments:
        await next(context)
        return
    print(f"Agent [{context.function.name}] called with messages: {context.arguments['messages']}")
    await next(context)
    print(f"Response from agent [{context.function.name}]: {context.result.value}")


# Create and configure the kernel.
kernel = Kernel()

# The filter is used for demonstration purposes to show the function invocation.
kernel.add_filter("function_invocation", function_invocation_filter)

thread: ChatHistoryAgentThread = None

In [3]:
# Define the CustomerBillDetails data type and mock plugin for billing data
class CustomerBillDetails(BaseModel):
    uuid: str
    bill_date: str
    due_date: str
    amount_due: float
    last_payment_date: str
    last_payment_amount: float

# Mock database for UUID to bill details mapping
uuid_to_bill: Dict[str, list[CustomerBillDetails]] = {}

class BillingDataPlugin:
    @kernel_function(description="Retrieves the most recent billing details for a customer UUID.")
    def get_bill_details(
        self, customer_uuid: Annotated[str, "The customer's UUID."]
    ) -> Annotated[CustomerBillDetails, "Returns the customer's most recent billing details as structured output."]:
        if customer_uuid not in uuid_to_bill or not uuid_to_bill[customer_uuid]:
            # Mock a list of bill details (simulate billing history)
            uuid_to_bill[customer_uuid] = [
                CustomerBillDetails(
                    uuid=customer_uuid,
                    bill_date="2024-04-01",
                    due_date="2024-04-15",
                    amount_due=89.99,
                    last_payment_date="2024-03-15",
                    last_payment_amount=89.99
                ),
                CustomerBillDetails(
                    uuid=customer_uuid,
                    bill_date="2024-03-01",
                    due_date="2024-03-15",
                    amount_due=89.99,
                    last_payment_date="2024-02-15",
                    last_payment_amount=89.99
                ),
                CustomerBillDetails(
                    uuid=customer_uuid,
                    bill_date="2024-02-01",
                    due_date="2024-02-15",
                    amount_due=79.99,
                    last_payment_date="2024-01-15",
                    last_payment_amount=89.99
                )
            ]
        # Return the most recent bill (first in the list)
        return uuid_to_bill[customer_uuid][0]

billing_settings=OpenAIChatPromptExecutionSettings()
billing_settings.response_format=CustomerBillDetails
billing_settings.tool_choice="auto"

billing_agent_name = "BillingAgent"
billing_agent = ChatCompletionAgent(
    service   = AzureChatCompletion(),
    name      = billing_agent_name,
    kernel    = kernel,
    instructions=(
        "You provide information about customer mobile bills such as amount due, "
        "due dates, payment methods, usage, and billing history.  "
        "You MUST call the `get_bill_details` function and return *only* the "
        "JSON matching the CustomerBillDetails schema."
    ),
    plugins   = [BillingDataPlugin()],
    arguments = KernelArguments(settings=billing_settings),
)

In [4]:
class RefundDetails(BaseModel):
    uuid: str
    refund_amount: float
    refund_date: str
    status: str

# Mock database for UUID to refund details mapping
uuid_to_refund: Dict[str, RefundDetails] = {}

def calculate_refund_amount(billed_amount: float, correct_amount: float) -> float:
    return max(0.0, billed_amount - correct_amount)

class RefundPlugin:
    @kernel_function(description="Calculates the refund amount and processes a refund for a customer UUID.")
    def process_refund(
        self,
        customer_uuid: Annotated[str, "The customer's UUID."],
        billed_amount: Annotated[float, "The amount the customer was billed."],
        correct_amount: Annotated[float, "The correct amount that should have been billed."]
    ) -> Annotated[RefundDetails, "Returns the refund details."]:
        refund_amount = calculate_refund_amount(billed_amount, correct_amount)
        refund = RefundDetails(
            uuid=customer_uuid,
            refund_amount=refund_amount,
            refund_date=datetime.now().strftime("%Y-%m-%d"),
            status="Processed" if refund_amount > 0 else "No refund needed"
        )
        uuid_to_refund[customer_uuid] = refund
        return refund

# Configure structured output format for refund agent
refund_settings = OpenAIChatPromptExecutionSettings()
refund_settings.tool_choice = "auto"
refund_settings.response_format = RefundDetails


refund_agent_name = "RefundAgent"
refund_agent = ChatCompletionAgent(
    service   = AzureChatCompletion(),
    kernel    = kernel,
    name      = "RefundAgent",
    instructions=(
        "You MUST call the `process_refund` function and return *only* the JSON argument "
        "object matching the `RefundDetails` schema — including the `customer_uuid` field.  "
        "Do not include any extra text."
    ),
    plugins   = [RefundPlugin()],
    arguments = KernelArguments(settings=refund_settings),
)

# Assign kernel to refund_agent
refund_agent.kernel = kernel

In [5]:
class CustomerAccountInfo(BaseModel):
    uuid: str
    phone_number: str
    plan_id: str

mock_accounts: Dict[str, CustomerAccountInfo] = {}

class AccountLookupPlugin:
    @kernel_function(description="Retrieves customer account info by phone number.")
    def get_account_info(
        self,
        phone_number: Annotated[str, "The customer's phone number."],
    ) -> Annotated[CustomerAccountInfo, "Structured customer account info."]:
        if phone_number not in mock_accounts:
            mock_accounts[phone_number] = CustomerAccountInfo(
                uuid=str(uuid.uuid4()),
                phone_number=phone_number,
                plan_id=f"plan_{phone_number[-4:]}"
            )
        return mock_accounts[phone_number]

# --- PlanDetailsPlugin: structured plan info ---
class PlanDetails(BaseModel):
    plan_id: str
    name: str
    data_limit_gb: int
    monthly_rate: float

mock_plans: Dict[str, PlanDetails] = {
    "plan_4567": PlanDetails(plan_id="plan_4567", name="Unlimited Plus", data_limit_gb=50, monthly_rate=79.99),
    "plan_1234": PlanDetails(plan_id="plan_1234", name="Basic", data_limit_gb=5, monthly_rate=29.99)
}

class PlanDetailsPlugin:
    @kernel_function(description="Retrieves plan details by plan_id.")
    def get_plan_details(
        self,
        plan_id: Annotated[str, "The plan ID."],
    ) -> Annotated[PlanDetails, "Structured plan details."]:
        return mock_plans.get(
            plan_id,
            PlanDetails(plan_id=plan_id, name="Unknown", data_limit_gb=0, monthly_rate=0.0)
        )

In [13]:
cs_settings = OpenAIChatPromptExecutionSettings()
cs_settings.tool_choice = "auto"
cs_settings.response_format = None

cs_agent = ChatCompletionAgent(
    service = AzureChatCompletion(),
    kernel  = kernel,
    name    = "CustomerServiceAgent",
    instructions=(
        "You are a customer‐service assistant for a mobile carrier.  "
        "Workflow:\n"
        "1. Lookup the customer’s UUID via AccountLookupPlugin\n"
        "2. Fetch plan details via PlanDetailsPlugin\n"
        "3. For billing questions, call BillingAgent.\n"
        "4. For refunds, use the exact UUID returned in step 1, get the billed amounts from BillingAgent, compare that with their plan details, "
        "then call the process_refund function (in RefundAgent) with JSON:\n"
        "{\"customer_uuid\":<uuid>,\"billed_amount\":<billed>,\"correct_amount\":<plan_rate>} "
        "and return *only* the JSON matching the RefundDetails schema.\n"
        "5. Summarize results clearly after the function call."
    ),
    plugins=[AccountLookupPlugin(), PlanDetailsPlugin(), billing_agent, refund_agent],
    arguments = KernelArguments(settings=cs_settings)
)

In [7]:
# Example user message
user_message = ChatMessageContent(
    role=AuthorRole.USER,
    name="customer",
    content="Hi, I was overcharged on a few bills and need a refund. My number is (555) 123-4567."
)

In [15]:
# Send the user message to the customer service agent and log all agent interactions
response = await cs_agent.get_response(
    messages=user_message,
    thread=thread,
)
print(f"Agent: {response}")

Agent [BillingAgent] called with messages: Retrieve the most recent bill details for customer UUID 1a686f2e-8ee4-4e18-960b-64cbdf60128a.
Response from agent [BillingAgent]: {"uuid":"1a686f2e-8ee4-4e18-960b-64cbdf60128a","bill_date":"2024-04-01","due_date":"2024-04-15","amount_due":89.99,"last_payment_date":"2024-03-15","last_payment_amount":89.99}
Agent: You were overcharged $10.00 on your most recent bill. A refund of $10.00 has been processed and the status is "Processed." If you have questions about other bills or need further assistance, please let me know!
